# PRACTISE DUCKDB

## QUICK START

In [2]:
import duckdb

connection = duckdb.connect()

In [6]:
# check how the output is shown
duckdb.sql("SELECT 42*8")

┌──────────┐
│ (42 * 8) │
│  int32   │
├──────────┤
│      336 │
└──────────┘

In [11]:
results = duckdb.sql("SELECT 42").fetchall()
print(results[0][0])
print(results)

42
[(42,)]


In [15]:
df = duckdb.sql("select 42").df()
df

,42
0,42


## PRACTISE

In [2]:
import duckdb
import pandas as pd

df = pd.read_csv("../data/currency_rate_local.csv")

In [4]:
df.head()

,timestamp,currency_code,exchange_rate
0,1690156800,EUR,0.914923
1,1690156800,GBP,0.782291
2,1690156800,INR,82.109338
3,1690156800,AUD,1.471667
4,1690156800,CAD,1.321980


In [8]:
duckdb.filter(df, "currency_code='EUR'")

┌────────────┬───────────────┬───────────────┐
│ timestamp  │ currency_code │ exchange_rate │
│   int64    │    varchar    │    double     │
├────────────┼───────────────┼───────────────┤
│ 1690156800 │ EUR           │      0.914923 │
└────────────┴───────────────┴───────────────┘